<a href="https://colab.research.google.com/github/MLandML/MLandML/blob/learning_projects/Spam_detection_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input,Embedding,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['spam.csv']),encoding='ISO-8859-1')
df.head()

In [ ]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

df.columns = ['label','data']
df.head()

In [ ]:
df['b_label'] = df['label'].map({'ham':0,'spam':1})
Y = df['b_label'].values
from sklearn.model_selection import train_test_split
df_train,df_test,y_train,y_test = train_test_split(df['data'],Y,test_size=0.33)

In [ ]:
from IPython.lib.pretty import MAX_SEQ_LENGTH
MAX_SEQ_LENGTH = 20000
tokenizer = Tokenizer(num_words=MAX_SEQ_LENGTH)
tokenizer.fit_on_texts(df_train)
sequence_train = tokenizer.texts_to_sequences(df_train)
sequence_test = tokenizer.texts_to_sequences(df_test) 

In [ ]:
word2idx = tokenizer.word_index
V = len(word2idx)

In [ ]:
data_train = pad_sequences(sequence_train)
T = data_train.shape[1]
data_test = pad_sequences(sequence_test,maxlen=T)

In [ ]:
D = 15
i = Input(shape=(T,))
x = Embedding(V+1,D)(i)
x = Conv1D(32,3,activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(64,3,activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128,3,activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1,activation='sigmoid')(x)

model = Model(i,x)

model.compile(
    loss = 'binary_crossentropy',
    optimizer = Adam(lr=0.01),
    metrics = ['accuracy']
)

r = model.fit(data_train,y_train,epochs=15,validation_data=(data_test,y_test))

In [ ]:
plt.plot(r.history['loss'],label='loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'],label='acc')
plt.plot(r.history['val_accuracy'],label='val_acc')
plt.legend()